In [1]:
import numpy as np
import matplotlib.pylab as plt
import glob
import time

In [2]:
import math
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, classes=10, kernel = 3, channels=3):
        super(Net, self).__init__()
        self.kernel = kernel
        self.channels = channels
        self.conv = nn.Conv2d(3,self.channels,self.kernel,1) 
        self.pad = math.ceil((32 - (32 - self.kernel + 1))/2)
        self.linear = nn.Linear(self.channels*(32 - self.kernel + 1 + self.pad*2)*(32 - self.kernel + self.pad*2 + 1),10)
        self.flat = nn.Flatten()

    def forward(self, x):
        x = torch.nn.functional.pad(x,(self.pad,self.pad,self.pad,self.pad),'circular')
        x = self.conv(x)
        x = self.flat(x)
        x = self.linear(x)
        return x
    
class Net3(nn.Module):
    def __init__(self, classes=10, kernel = 3, channels=3):
        super(Net3, self).__init__()
        self.kernel = kernel
        self.channels = channels
        
        self.conv1 = nn.Conv2d(3,self.channels,self.kernel,1) 
        self.pad1 = math.ceil((32 - (32 - self.kernel + 1))/2)
        self.output1 = int((32 - self.kernel + 1 + self.pad1*2))
        
        self.conv2 = nn.Conv2d(self.channels, self.channels, self.kernel,1) 
        self.pad2 = math.ceil((self.output1 - (self.output1 - self.kernel + 1))/2)
        self.output2 = int((self.output1 - self.kernel + 1 + self.pad2*2))
        
        self.conv3 = nn.Conv2d(self.channels, self.channels, self.kernel,1) 
        self.pad3 = math.ceil((self.output2 - (self.output2 - self.kernel + 1))/2)
        self.output3 = int((self.output2 - self.kernel + 1 + self.pad3*2))
        
        self.linear = nn.Linear(self.channels*(self.output3 - self.kernel + 1 + self.pad3*2)*(self.output3 - self.kernel + self.pad3*2 + 1),10)
        self.flat = nn.Flatten()

    def forward(self, x):
        x = torch.nn.functional.pad(x,(self.pad1,self.pad1,self.pad1,self.pad1),'circular')
        x = self.conv1(x)
        x = torch.nn.functional.pad(x,(self.pad2,self.pad2,self.pad2,self.pad2),'circular')
        x = self.conv2(x)
        x = torch.nn.functional.pad(x,(self.pad3,self.pad3,self.pad3,self.pad3),'circular')
        x = self.conv3(x)
        x = self.flat(x)
        x = self.linear(x)
        return x
    


posible_models = ['fclinearl3', 'fclinearl1','convlinearl1k3c3', 'convlinearl3k3c3','convlinearl1k11c3','convlinearl3k11c3','convlinearl1k32c3',
                 'convlinearl3k32c3','convlinearl1k3c8','convlinearl1k3c32','convlinearl3k3c8','convlinearl3k3c32']

def return_model(name='fclinearl1'):
    if name == 'fclinearl3':
        model = nn.Sequential(nn.Flatten(), nn.Linear(3*32*32,3*32*32),nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,10))
    elif name == 'fclinearl1':
        model = nn.Sequential(nn.Flatten(),nn.Linear(3*32*32,3*32*32), nn.Linear(3*32*32,10))
    elif name == 'convlinearl1k3c3':
        model = Net(kernel=3,channels=3)
    elif name == 'convlinearl3k3c3':
        model = Net3(kernel=3,channels=3)
    elif name == 'convlinearl1k11c3':
        model = Net(kernel=11,channels=3)
    elif name == 'convlinearl3k11c3':
        model = Net3(kernel=11, channels=3)
    elif name == 'convlinearl1k32c3':
        model = Net(kernel=32, channels=3)
    elif name == 'convlinearl3k32c3':
        model = Net3(kernel=32, channels=3)
    elif name == 'convlinearl1k3c8':
        model = Net(kernel=3,channels=8)
    elif name == 'convlinearl1k3c32':
        model = Net(kernel=3, channels=32)
    elif name == 'convlinearl3k3c8':
        model = Net3(kernel=3, channels=8)
    elif name == 'convlinearl3k3c32':
        model = Net3(kernel=3, channels=32)
        
    return model

In [6]:
def rgb2gray(rgb):

    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def get_fourier(imag):
    image_sample = np.fft.fft2(imag)
    F_orig = np.fft.fftshift(image_sample)
    F_orig = np.log(np.abs(F_orig))
    return F_orig

model = 'convlinearl1k3c3'
import time
import scipy.spatial
image_total = []
delta_total = []
distance_total = []
fourier_total = []
maximum_total = []
original_total = []
adversarial_total = []
normal_total = []
saliency_total = []
sfourier_total = []
m = time.time()
namess = './cifar_resnet/ckpt_{}_*_*_linf_adversarial_100.npy'.format(model)
namess = glob.glob(namess)
maxss = []
for name in namess:
    name = name.split('/')[-1].split('_')[2]
    maxss.append(int(name))
maxs = max(maxss)
mins = min(maxss)
for i in range(mins,maxs+1):
    for h in range(1,392,30):
        name = './cifar_resnet/ckpt_{}_{}_{}_linf_adversarial_100.npy'.format(model, i,h)
        name_s = './cifar_resnet/ckpt_{}_{}_{}_linf_saliency_100.npy'.format(model, i,h)
        b = np.load(name, allow_pickle=True)
        b = b[0,:,:]
        b1 = np.load(name_s, allow_pickle=True)
        images = []
        deltas = []
        distances = []
        fouriers = []
        maximums = []
        originals = []
        adversarials = []
        normals = []
        saliencys = []
        sfouriers = []
        for j in range(b.shape[0]):
            if b[j,3] != None:
                normals.append(b[j,1])
                delta = b[j,0] - b[j,1]
                delta = rgb2gray(delta)
                deltas.append(delta)
                fourier = get_fourier(delta)
                fouriers.append(fourier)
                maximums.append(fourier.max())
                original = b[j,1].flatten()
                adversarial = b[j,0].flatten()
                distance = scipy.spatial.distance.euclidean(original, adversarial)
                images.append(b[j,0])
                distances.append(distance)
                originals.append(b[j,2])
                adversarials.append(b[j,3])
                saliencys.append(b1[j,...])
                sfour = get_fourier(b1[j,...].mean(axis=0))
                sfouriers.append(sfour)
            else:
                normals.append(np.zeros((3,32,32)))
                delta = np.zeros((32,32))
                deltas.append(delta)
                images.append(b[j,1])
                distances.append(None)
                fouriers.append(delta)
                maximums.append(None)
                originals.append(b[j,2])
                adversarials.append(None)
                saliencys.append(b1[j,...])
                sfour = get_fourier(b1[j,...].mean(axis=0))
                sfouriers.append(sfour)

        
        images = np.array(images)
        deltas = np.array(deltas)
        normals = np.array(normals)
        distances = np.array(distances)
        fouriers = np.array(fouriers)
        maximums = np.array(maximums)
        originals = np.array(originals)
        adversarials = np.array(adversarials)
        saliencys = np.array(saliencys)
        sfouriers = np.array(sfouriers)
        image_total.append(images)
        delta_total.append(deltas)
        distance_total.append(distances)
        fourier_total.append(fouriers)
        maximum_total.append(maximums)
        original_total.append(originals)
        adversarial_total.append(adversarials)
        normal_total.append(normals)
        saliency_total.append(saliencys)
        sfourier_total.append(sfouriers)
        if (h) %25 ==0:
            print(i, j, time.time() - m)  
            m = time.time()
    
image_total = np.array(image_total)
delta_total = np.array(delta_total)
distance_total = np.array(distance_total)
fourier_total = np.array(fourier_total)
maximum_total = np.array(maximum_total)
original_total = np.array(original_total)
normal_total = np.array(normal_total)
adversarial_total = np.array(adversarial_total)
saliency_total = np.array(saliency_total)
sfourier_total = np.array(sfourier_total)

np.save('./cifar_resnet/cifar_dynamics_adv_normal_{}.npy'.format(model), normal_total)
np.save('./cifar_resnet/cifar_dynamics_adv_image_{}.npy'.format(model), image_total)
np.save('./cifar_resnet/cifar_dynamics_adv_delta_{}.npy'.format(model), delta_total)
np.save('./cifar_resnet/cifar_dynamics_adv_distance_{}.npy'.format(model), distance_total)
np.save('./cifar_resnet/cifar_dynamics_adv_fourier_{}.npy'.format(model), fourier_total)
np.save('./cifar_resnet/cifar_dynamics_adv_maximum_{}.npy'.format(model), maximum_total)
np.save('./cifar_resnet/cifar_dynamics_adv_original_{}.npy'.format(model), original_total)
np.save('./cifar_resnet/cifar_dynamics_adv_adversarial_{}.npy'.format(model), adversarial_total)
np.save('./cifar_resnet/cifar_dynamics_adv_saliency_{}.npy'.format(model), saliency_total)
np.save('./cifar_resnet/cifar_dynamics_adv_sfourier_{}.npy'.format(model), sfourier_total)


In [ ]:
%%capture
import imageio
import io
import cv2
import random
#image_total, delta_total, distance_total, fourier_total, maximum_total, original_total, adversarial_total
import matplotlib.gridspec as gridspec
import time

def get_img_from_fig(fig, dpi=100):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img


def dyn_fourier(normal, image, delta, distance, fourier, maximum, original, adversarial, saliency, sfourier, idx, i):
    fig = plt.figure(figsize=(16, 16))
    #fig, axs = plt.subplots(8,5, figsize=(40, 20), facecolor='w', edgecolor='k')
    gs1 = gridspec.GridSpec(nrows=8, ncols=6, left=0.05, right=1.0,
    wspace=0.5, hspace=0.5)
    
    for j in range(8):
        axs = fig.add_subplot(gs1[j, 0])
        axs.imshow(np.moveaxis(normal[idx[j],...], 0, -1), vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))
        
        axs = fig.add_subplot(gs1[j, 1])
        axs.imshow(np.moveaxis(image[idx[j],...], 0, -1), vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 2])
        axs.imshow(delta[idx[j],...], interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 3])
        axs.imshow(fourier[idx[j],...],vmin=0.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 4])
        axs.imshow(np.moveaxis(saliency[idx[j],...], 0, -1), vmin=0.0, vmax=1.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))

        axs = fig.add_subplot(gs1[j, 5])
        axs.imshow(sfourier[idx[j],...],vmin=0.0, interpolation='nearest')
        axs.set_title('E:{}, I:{}, C:{}. A:{}'.format(int(i // 12), int(i % 13)*30 +1,original[idx[j]], adversarial[idx[j]]))
    fig.suptitle('{}'.format(model), fontsize=12)
    image = get_img_from_fig(fig)
    del fig
    return image

vmax = image_total.max()
#vmin = image_total.min()
kargs = { 'macro_block_size': None }
idx = [i for i in range(image_total[0].shape[0])]
#random.shuffle(idx)
imageio.mimsave('./image_total_dynamics_cifar_{}.mp4'.format(model), [dyn_fourier(normal_total[i,...], image_total[i,...], delta_total[i,...], 
                                                               distance_total, fourier_total[i,...], 
                                                               maximum_total, original_total[i,...], adversarial_total[i,...], saliency_total[i,...],sfourier_total[i,...],idx,i) 
                                                   for i in range(image_total.shape[0])], fps=60, **kargs)